# Limpieza del conjunto de datos ATUS

En esta libreta se desarrolla el proceso de **limpieza y decodificación de variables** del conjunto de datos **Accidentes de Tránsito Terrestre en Zonas Urbanas y Suburbanas (ATUS)**, publicado por el **INEGI**. Este conjunto contiene información sobre los accidentes viales registrados en México, incluyendo fecha, ubicación geográfica, tipo de accidente, causas probables, características del conductor y tipo de vía, entre otros atributos. 

El flujo de trabajo relacionado con estos datos se compone de tres etapas principales: 

* Descarga: `src/download_atus.py` - Descarga los archivos fuente desde la página oficail del INEGI. 

* Extracción: `src/extract_atus.py` - Extrae y organiza los archivos descargados.
    
    * Ingregración: `notebooks/1.1-extraccion-datos-atus.opynb` - Describe el uso y validación de los scripts anteriores, mmostrando el flujo de extracción paso a paso. 


En la presente libreta se continua con el proceso aplicando tareas de limpieza de los datos extraidos, enfocándose en: 

* Integración de archivos CSV individuales en un único archivo. 
* Conversion de variables numéricas a categoricas usando el diccionario de datos.
* Eliminación de columnas irrelevantes y correccion de nombres de variables. 

Para realizar la decodificación de variables categoricas se utilizará el **diccionario de datos oficial** proporcionado por el INEGI que puede ser encontrado en la ruta (***). Este diccionario especifica los nombres de campo, tipos de dato, ranogs y significados de cada variable. 

## Configuración del entorno

A continuación se importan las funciones y rutas necesarias para proceder coon la limpieza de los datos ATUS:  

In [1]:
import sys
from pathlib import Path

import pandas as pd
import geopandas as gpd

In [2]:
SRC_DIR = Path().resolve().parent.parent / "src"

if str(SRC_DIR) not in sys.path: 
    sys.path.insert(0, str(SRC_DIR))

In [3]:
from config import ROOT_DIR, RAW_DIR, INTERIM_DIR
from extract_atus import INTERIM_ATUS_DIR

Antes de comenzar con la limpieza de los datos, verificamos que las rutas de almacenamiento no estén vacías:

In [4]:
for item in INTERIM_ATUS_DIR.iterdir(): 
    print(item.relative_to(ROOT_DIR))

data\interim\atus\BASE MUNICIPAL_ACCIDENTES DE TRANSITO GEORREFERENCIADOS_2021_HMO.csv
data\interim\atus\BASE MUNICIPAL_ACCIDENTES DE TRANSITO GEORREFERENCIADOS_2022_HMO.csv
data\interim\atus\BASE MUNICIPAL_ACCIDENTES DE TRANSITO GEORREFERENCIADOS_2023_HMO.csv


Si los datos están presentes, podemos cargar las rutas de los archivos CSV de la siguiente manera:

In [5]:
def get_all_csvs(path_base=INTERIM_ATUS_DIR):
    return [item for item in path_base.rglob("*.csv") if item.is_file()]

path_csvs = get_all_csvs(INTERIM_ATUS_DIR)

for csv in path_csvs: print(csv.relative_to(ROOT_DIR))

data\interim\atus\BASE MUNICIPAL_ACCIDENTES DE TRANSITO GEORREFERENCIADOS_2021_HMO.csv
data\interim\atus\BASE MUNICIPAL_ACCIDENTES DE TRANSITO GEORREFERENCIADOS_2022_HMO.csv
data\interim\atus\BASE MUNICIPAL_ACCIDENTES DE TRANSITO GEORREFERENCIADOS_2023_HMO.csv


* **Nota**: Si el directorio `data/interim/atus` está vacío, es necesario ejecutar el siguiente comando desde el directorio `src`:

```bash
python download_atus.py && python extract_atus.py
```

Este comando descargará los datos, filtrará los registros de Hermosillo y guardará el resultado en `data/interim/atus`. 



## Carga y concatenación de archivos

Primero, creamos una lista de DataFrames leyendo todos los archivos CSV encontrados:


In [6]:
dfs = [pd.read_csv(path_csv, index_col=0) for path_csv in path_csvs]

Luego, concatenamos todos los DataFrames en uno solo:

In [7]:
df = pd.concat(dfs)

Ahora, verificamos información general del DataFrame:

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33893 entries, 173098 to 236734
Data columns (total 50 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ID          33893 non-null  object 
 1   EDO         33893 non-null  int64  
 2   MES         33893 non-null  int64  
 3   ANIO        33893 non-null  int64  
 4   MPIO        33893 non-null  int64  
 5   HORA        33893 non-null  int64  
 6   MINUTOS     33893 non-null  int64  
 7   DIA         33893 non-null  int64  
 8   DIASEMANA   33893 non-null  int64  
 9   URBANA      33893 non-null  int64  
 10  SUBURBANA   33893 non-null  int64  
 11  TIPACCID    33893 non-null  int64  
 12  AUTOMOVIL   33893 non-null  int64  
 13  CAMPASAJ    33893 non-null  int64  
 14  MICROBUS    33893 non-null  int64  
 15  PASCAMION   33893 non-null  int64  
 16  OMNIBUS     33893 non-null  int64  
 17  TRANVIA     33893 non-null  int64  
 18  CAMIONETA   33893 non-null  int64  
 19  CAMION      33893 non-nu

In [9]:
df.head()

,ID,EDO,MES,ANIO,MPIO,HORA,MINUTOS,DIA,DIASEMANA,URBANA,...,OTROHERIDO,TOTMUERTOS,TOTHERIDOS,CLASE,CALLE1,CALLE2,CARRETERA,LONGITUD,LATITUD,OID
173098,1006278-999-0,26,1,2021,30,5,0,1,5,0,...,0,0,1,2,HERMOSILLO - BAHÍA KINO,CARRETERA 100,HERMOSILLO - BAHÍA KINO,-111.175579,29.005638,NaN
173099,1006278-999-1,26,1,2021,30,14,55,1,5,1,...,0,0,0,3,LEY FEDERAL DEL TRABAJO,ARIZONA,NaN,-110.977345,29.118933,NaN
173100,1006278-999-2,26,1,2021,30,15,20,1,5,1,...,0,0,0,3,SOLIDARIDAD,RICARDO VALENCIA Y SOUZA,NaN,-110.994506,29.135785,NaN
173101,1006278-999-3,26,1,2021,30,22,30,1,5,1,...,0,0,0,3,TERCERA DE WISTARIA,LÁZARO MERCADO,NaN,-111.016417,29.124394,NaN
173102,1006278-999-5,26,1,2021,30,8,45,2,6,0,...,0,0,0,3,HERMOSILLO - BAHÍA KINO,NaN,HERMOSILLO - BAHÍA KINO,-111.115522,29.050191,NaN


## Limpieza

Con lo anterior, procedemos a limpiar el conjunto de datos. 

El índice original no es útil, podemos eliminarlo y reemplazarlo por uno generico: 

In [10]:
df.reset_index(inplace=True, drop=True)
df.head()

,ID,EDO,MES,ANIO,MPIO,HORA,MINUTOS,DIA,DIASEMANA,URBANA,...,OTROHERIDO,TOTMUERTOS,TOTHERIDOS,CLASE,CALLE1,CALLE2,CARRETERA,LONGITUD,LATITUD,OID
0,1006278-999-0,26,1,2021,30,5,0,1,5,0,...,0,0,1,2,HERMOSILLO - BAHÍA KINO,CARRETERA 100,HERMOSILLO - BAHÍA KINO,-111.175579,29.005638,NaN
1,1006278-999-1,26,1,2021,30,14,55,1,5,1,...,0,0,0,3,LEY FEDERAL DEL TRABAJO,ARIZONA,NaN,-110.977345,29.118933,NaN
2,1006278-999-2,26,1,2021,30,15,20,1,5,1,...,0,0,0,3,SOLIDARIDAD,RICARDO VALENCIA Y SOUZA,NaN,-110.994506,29.135785,NaN
3,1006278-999-3,26,1,2021,30,22,30,1,5,1,...,0,0,0,3,TERCERA DE WISTARIA,LÁZARO MERCADO,NaN,-111.016417,29.124394,NaN
4,1006278-999-5,26,1,2021,30,8,45,2,6,0,...,0,0,0,3,HERMOSILLO - BAHÍA KINO,NaN,HERMOSILLO - BAHÍA KINO,-111.115522,29.050191,NaN


Las columnas se encuentran en mayúsculas, las transoformamos a minúsculas: 

In [11]:
df.columns = df.columns.str.lower()
df.head()

,id,edo,mes,anio,mpio,hora,minutos,dia,diasemana,urbana,...,otroherido,totmuertos,totheridos,clase,calle1,calle2,carretera,longitud,latitud,oid
0,1006278-999-0,26,1,2021,30,5,0,1,5,0,...,0,0,1,2,HERMOSILLO - BAHÍA KINO,CARRETERA 100,HERMOSILLO - BAHÍA KINO,-111.175579,29.005638,NaN
1,1006278-999-1,26,1,2021,30,14,55,1,5,1,...,0,0,0,3,LEY FEDERAL DEL TRABAJO,ARIZONA,NaN,-110.977345,29.118933,NaN
2,1006278-999-2,26,1,2021,30,15,20,1,5,1,...,0,0,0,3,SOLIDARIDAD,RICARDO VALENCIA Y SOUZA,NaN,-110.994506,29.135785,NaN
3,1006278-999-3,26,1,2021,30,22,30,1,5,1,...,0,0,0,3,TERCERA DE WISTARIA,LÁZARO MERCADO,NaN,-111.016417,29.124394,NaN
4,1006278-999-5,26,1,2021,30,8,45,2,6,0,...,0,0,0,3,HERMOSILLO - BAHÍA KINO,NaN,HERMOSILLO - BAHÍA KINO,-111.115522,29.050191,NaN


Las columnas `edo` y `mio` corresponden al estado y municipio respectivamente. Como los datos corresponden a Hermosillo, Sonora es redundante que estén presentes en el DataFrame. Podemos eliminarlas: 

In [12]:
df.drop(columns=['edo', 'mpio'], inplace=True)

Dado que tenemos columnas separadas para el año, mes, día, hora y minutos en los que ocurrió cada accidente, podemos combinarlas en una sola columna de tipo *timestamp*: 

In [13]:
df['datetime'] = pd.to_datetime(
    df['anio'].astype(str) + '-' +
    df['mes'].astype(str).str.zfill(2) + '-' +
    df['dia'].astype(str).str.zfill(2) + ' ' +
    df['hora'].astype(str).str.zfill(2) + ':' +
    df['minutos'].astype(str).str.zfill(2)
)
df.datetime.head()

0   2021-01-01 05:00:00
1   2021-01-01 14:55:00
2   2021-01-01 15:20:00
3   2021-01-01 22:30:00
4   2021-01-02 08:45:00
Name: datetime, dtype: datetime64[ns]

### Decodificación

En lo que sigue se exploran las variables categóricas del conjunto de datos. El objetivo es **reempplazar sus valores numéricos por etiquetas descriptivas** seg´un el diccionario deatos proporcionado por el INEGI disponible en la ruta (***). 

Para cada variable se implementa una función que convierte la vairbale numérica en categorica usando el diccionario de datos y se prueba su funcionamiento. 

La variable `diasemana` reprecenta el día de la semana en la que ocurrio el accidente:

In [14]:
df.diasemana.unique()

array([5, 6, 7, 1, 2, 3, 4])

In [ ]:
def decode_dia_semana(valor):
    dias = {
        1: "lunes", 2: "martes", 3: "miércoles", 4: "jueves",
        5: "viernes", 6: "sábado", 7: "domingo"
    }
    return dias.get(valor)

In [16]:
df['diasemana'] = df['diasemana'].apply(decode_dia_semana)
df.diasemana.unique()

array(['viernes', 'sábado', 'domingo', 'lunes', 'martes', 'miércoles',
       'jueves'], dtype=object)

La variable `urbana` indica el tipo de zona urbana donde ocurrió el accidente:

Según el diccionario de datos: 

* 0 = Zona suburbana
* 1 = Intersección
* 2 = No intersección

In [17]:
df.urbana.unique()

array([0, 1, 2])

In [ ]:
def decode_zona_urbana(valor):
    zonas = {0: "suburbana", 1: "intersección", 2: "no intersección"}
    return zonas.get(valor)

In [19]:
df['urbana'] = df['urbana'].apply(decode_zona_urbana)
df.urbana.unique()

array(['suburbana', 'intersección', 'no intersección'], dtype=object)

La variable `suburbana` describe el tipo de zona suburbana del accidente.

Según el diccionario:

* 0 = Urbana
* 1 = Camino rural
* 2 = Carretera estatal
* 3 = Otro camino

In [20]:
df.suburbana.unique()

array([2, 0, 1])

In [21]:
def decode_zona_suburbana(valor):
    zonas = {
        0: "urbana", 1: "camino rural",
        2: "carretera estatal", 3: "otro camino"
    }
    return zonas.get(valor)

In [22]:
df['suburbana'] = df['suburbana'].apply(decode_zona_suburbana)
df.suburbana.unique()

array(['carretera estatal', 'urbana', 'camino rural'], dtype=object)

La variable `tipaccid` identifica el tipo de accidente registrado.

De acuerdo con el diccionario:

* 1 = Colisión con vehículo automotor
* 2 = Atropellamiento
* 3 = Colisión con animal
* 4 = Colisión con objeto fijo
* 5 = Volcadura
* 6 = Caída de pasajero
* 7 = Salida del camino
* 8 = Incendio
* 9 = Colisión con ferrocarril
* 10 = Colisión con motocicleta
* 11 = Colisión con ciclista
* 12 = Otro


In [23]:
df.tipaccid.unique()

array([ 3,  1,  4, 10,  2,  5, 11,  6,  7, 12,  9])

In [24]:
def decode_tipo_accidente(valor):
    tipos = {
        0: "certificado cero",
        1: "colisión con vehículo automotor",
        2: "atropellamiento",
        3: "colisión con animal",
        4: "colisión con objeto fijo",
        5: "volcadura",
        6: "caída de pasajero",
        7: "salida del camino",
        8: "incendio",
        9: "colisión con ferrocarril",
        10: "colisión con motocicleta",
        11: "colisión con ciclista",
        12: "otro"
    }
    return tipos.get(valor)

In [25]:
df['tipaccid'] = df['tipaccid'].apply(decode_tipo_accidente)
df.tipaccid.unique()

array(['colisión con animal', 'colisión con vehículo automotor',
       'colisión con objeto fijo', 'colisión con motocicleta',
       'atropellamiento', 'volcadura', 'colisión con ciclista',
       'caída de pasajero', 'salida del camino', 'otro',
       'colisión con ferrocarril'], dtype=object)

La variable `causaacci` indica la causa probable o presunta del accidente.

Según el diccionario:

* 1 = Conductor
* 2 = Peatón o pasajero
* 3 = Falla del vehículo
* 4 = Mala condición del camino
* 5 = Otra


In [26]:
df.causaacci.unique()

array([1, 2, 3])

In [27]:
def decode_causa_accidente(valor):
    causas = {
        1: "conductor", 2: "peatón/pasajero",
        3: "falla del vehículo", 4: "mala condición del camino", 5: "otra"
    }
    return causas.get(valor)

In [28]:
df['causaacci'] = df['causaacci'].apply(decode_causa_accidente)
df.causaacci.unique()

array(['conductor', 'peatón/pasajero', 'falla del vehículo'], dtype=object)

La variable `caparod` señala la condición de la superficie del camino.

Valores posibles:

* 1 = Pavimentada
* 2 = No pavimentada



In [29]:
df.caparod.unique()

array([1, 2])

In [30]:
def decode_capa_rodamiento(valor):
    capas = {1: "pavimentada", 2: "no pavimentada"}
    return capas.get(valor)

In [31]:
df['caparod'] = df['caparod'].apply(decode_capa_rodamiento)
df.caparod.unique()

array(['pavimentada', 'no pavimentada'], dtype=object)

La variable `sexo` representa el sexo del conductor presunto responsable.

Valores:

* 1 = Se fugó
* 2 = Hombre
* 3 = Mujer



In [32]:
df.sexo.unique()

array([2, 3, 1])

In [33]:
def decode_sexo(valor):
    sexos = {1: "se fugó", 2: "hombre", 3: "mujer"}
    return sexos.get(valor)

In [34]:
df['sexo'] = df['sexo'].apply(decode_sexo)
df.sexo.unique()

array(['hombre', 'mujer', 'se fugó'], dtype=object)

La variable `aliento` indica si el conductor presentaba aliento alcohólico.

Según el diccionario:

* 4 = Sí
* 5 = No
* 6 = Se ignora

In [35]:
df.aliento.unique()

array([5, 6, 4])

In [36]:
def decode_aliento(valor):
    aliento = {4: "sí", 5: "no", 6: "se ignora"}
    return aliento.get(valor)

In [37]:
df['aliento'] = df['aliento'].apply(decode_aliento)
df.aliento.unique()

array(['no', 'se ignora', 'sí'], dtype=object)

La variable `cinturon` representa el uso del cinturón de seguridad.

De acuerdo con el diccionario:

* 7 = Sí
* 8 = No
* 9 = Se ignora

In [39]:
df.cinturon.unique()

array([8, 7, 9])

In [ ]:
def decode_cinturon(valor):
    cinturon = {7: "sí", 8: "no", 9: "se ignora"}
    return cinturon.get(valor)

In [41]:
df['cinturon'] = df['cinturon'].apply(decode_cinturon)
df.cinturon.unique()

array(['no', 'sí', 'se ignora'], dtype=object)

La variable `clase` clasifica la gravedad del accidente.

Valores posibles:

* 1 = Fatal
* 2 = No fatal
* 3 = Solo daños

In [42]:
df.clase.unique()

array([2, 3, 1])

In [43]:
def decode_clase_accidente(valor):
    clases = {1: "fatal", 2: "no fatal", 3: "solo daños"}
    return clases.get(valor)

In [44]:
df['clase'] = df['clase'].apply(decode_clase_accidente)
df.clase.unique()

array(['no fatal', 'solo daños', 'fatal'], dtype=object)

El siguiente paso es formalizar las transformaciones aquí descritas dentro del un scrip de limpieza, `src/clean_atus.py`, donde se integrarán las funciones de limpieza y decodificación del conjunto de datos. De esta forma, el proceso podrá ejecutarse de manera automatizada y reproducible, ya sea desde la terminal o como parte del flujo de procesamiento. 